In [1]:
import glob
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from prody import *
import openpyxl

In [6]:
data_path = '/opt/experiments/pep_hla_dynamics/data_compiled/'
int_en_files = glob.glob(data_path+'/*/*/EC_Data/*/energies_intEnTotal.csv')
int_en_files_vdw = glob.glob(data_path+'/*/*/EC_Data/*/energies_intEnVdW.csv')

In [4]:
df_metadata = pd.DataFrame(columns=['subtype','peptide','filename'])

i = 0
for fl in int_en_files_vdw:
    matches = re.search('.*EC_Data/(HLA_.*)_(.*)/energies_.*', fl)
    if matches:
        subtype_name = matches.groups()[0]
        peptide = matches.groups()[1]
        df_metadata.loc[i,'filename'] = fl
        df_metadata.loc[i,'subtype'] = subtype_name
        df_metadata.loc[i,'peptide'] = peptide
        i += 1

In [5]:
df_ie_list = list()

for row in df_metadata.iterrows():
    df = pd.read_csv(row[1]['filename'])
    syst = parsePDB('/'.join(row[1]['filename'].split('/')[:-1])+'/system.pdb')
    df = df.transpose()

    for row_df in df.iterrows():
        if row_df[0].startswith('Unnamed: '):
            continue
        pair = row_df[0].split('-')
        res1 = int(pair[0])
        res2 = int(pair[1])
        df.loc[row_df[0],'res1_index'] = res1
        df.loc[row_df[0],'res2_index'] = res2
        chid1 = syst.select(f'resindex {res1}').getChids()[0]
        chid2 = syst.select(f'resindex {res2}').getChids()[0]
        resnum1 = syst.select(f'resindex {res1}').getResnums()[0]
        resnum2 = syst.select(f'resindex {res2}').getResnums()[0]
        df.loc[row_df[0],'res1_chid'] = chid1
        df.loc[row_df[0],'res2_chid'] = chid2
        df.loc[row_df[0],'res1_resnum'] = resnum1
        df.loc[row_df[0],'res2_resnum'] = resnum2

    df.dropna(inplace=True)
    df.reset_index(inplace=True)
    df.rename(columns={0: 'IE'}, inplace=True)
    df.drop(['index'], axis=1, inplace=True)
    df['subtype'] = row[1]['subtype']
    df['peptide'] = row[1]['peptide']

    df_ie_list.append(df)
    
df_ie_master = pd.concat(df_ie_list,axis=0, ignore_index = False)

@> 6101 atoms and 1 coordinate set(s) were parsed in 0.04s.


@> 6088 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 6114 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 6131 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 6097 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 6106 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 6090 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 6116 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 6100 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 6095 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 6096 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 6107 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 6070 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 6097 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 6090 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 6120 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 6120 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 6068 atoms and 1 coordinate set(s) we

In [7]:
df_ie_master.to_csv(data_path+'energies_intEnVdW_master.csv', index=False)

In [8]:
df_ie_master.to_excel(data_path+"df_ie_master_vdw.xlsx") 